# Package loading and basic configurations

In [4]:
%load_ext autoreload
%autoreload 2

# load dependencies'
import pandas as pd
import geopandas as gpd

from envirocar import TrackAPI, DownloadClient, BboxSelector, ECConfig, TimeSelector

# create an initial but optional config and an api client
config = ECConfig()
track_api = TrackAPI(api_client=DownloadClient(config=config))

# Querying enviroCar Tracks

In [5]:
bbox = BboxSelector([
    7.615825, # min_x
    51.954942, # min_y
    7.632137, # max_x
    51.968326  # max_y
])

time_interval = TimeSelector(
    '2019-01-01T00:00:00Z',
    '2019-12-31T00:00:00Z'
)

# issue a query
track_df = track_api.get_tracks(bbox=bbox, num_results=10, time_interval=time_interval)# requesting 10 tracks inside the bbox
track_df

,id,time,geometry,GPS VDOP.value,GPS VDOP.unit,GPS Speed.value,GPS Speed.unit,Intake Temperature.value,Intake Temperature.unit,CO2.value,...,sensor.constructionYear,sensor.manufacturer,track.appVersion,track.touVersion,O2 Lambda Voltage ER.value,O2 Lambda Voltage ER.unit,O2 Lambda Voltage.value,O2 Lambda Voltage.unit,MAF.value,MAF.unit
0,5e10ec059115b85a12318e64,2019-12-30T09:31:20,POINT (7.59969 51.93433),0.800000,precision,9.459943,km/h,28.000000,c,13.898474,...,2007,Dodge,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5e10ec059115b85a12318e66,2019-12-30T09:31:26,POINT (7.59983 51.93449),0.851000,precision,18.669282,km/h,28.000000,c,4.336944,...,2007,Dodge,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5e10ec059115b85a12318e67,2019-12-30T09:31:31,POINT (7.60002 51.93457),0.800000,precision,4.359976,km/h,28.000001,c,4.170298,...,2007,Dodge,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5e10ec059115b85a12318e68,2019-12-30T09:31:36,POINT (7.60003 51.93458),0.832303,precision,0.000000,km/h,28.000000,c,4.100011,...,2007,Dodge,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5e10ec059115b85a12318e69,2019-12-30T09:31:41,POINT (7.60003 51.93458),0.827140,precision,0.000000,km/h,27.999999,c,4.074120,...,2007,Dodge,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,5d222f4644ea855023c1d1de,2019-07-07T17:00:47,POINT (7.59671 51.99927),2.227546,precision,29.906211,km/h,51.000000,c,7.016605,...,2003,Volvo,NaN,NaN,NaN,NaN,NaN,NaN,9.083020,l/s
284,5d222f4644ea855023c1d1df,2019-07-07T17:00:52,POINT (7.59628 51.99961),1.800805,precision,40.760203,km/h,50.000001,c,9.879350,...,2003,Volvo,NaN,NaN,NaN,NaN,NaN,NaN,12.788853,l/s
285,5d222f4644ea855023c1d1e0,2019-07-07T17:00:58,POINT (7.59577 52.00007),2.500000,precision,42.986743,km/h,51.000001,c,6.076898,...,2003,Volvo,NaN,NaN,NaN,NaN,NaN,NaN,7.866566,l/s
286,5d222f4644ea855023c1d1e1,2019-07-07T17:01:03,POINT (7.59525 52.00050),2.500000,precision,39.836086,km/h,50.999999,c,2.060684,...,2003,Volvo,NaN,NaN,NaN,NaN,NaN,NaN,2.667563,l/s


In [ ]:
track_df.plot(figsize=(8, 10))

# Presenting Summary Statistics

In [ ]:
#Using .describe() fuction to generate descriptive statistics about the dataframe

track_df.describe()

# Inspecting a single Track

In [ ]:
some_track_id = track_df['track.id'].unique()[5]
some_track = track_df[track_df['track.id'] == some_track_id]
some_track.plot()

In [ ]:
ax = some_track['Speed.value'].plot()
ax.set_title("Speed")
ax.set_ylabel(some_track['Speed.unit'][0])
ax

## Interactive Map
The following map-based visualization makes use of folium. It allows to visualizate geospatial data based on an interactive leaflet map. Since the data in the GeoDataframe is modelled as a set of Point instead of a LineString, we have to manually create a polyline

In [ ]:
import folium

lats = list(some_track['geometry'].apply(lambda coord: coord.y))
lngs = list(some_track['geometry'].apply(lambda coord: coord.x))

avg_lat = sum(lats) / len(lats)
avg_lngs = sum(lngs) / len(lngs)

m = folium.Map(location=[avg_lat, avg_lngs], zoom_start=13)
folium.PolyLine([coords for coords in zip(lats, lngs)], color='blue').add_to(m)
m

# Example: Visualization with pydeck (deck.gl)

The pydeck library makes use of the basemap tiles from Mapbox. In case you want to visualize the map with basemap tiles, you need to register with MapBox, and configure a specific access token. The service is free until a certain level of traffic is esceeded.

You can either configure it via your terminal (i.e. `export MAPBOX_API_KEY=<mapbox-key-here>`), which pydeck will automatically read, or you can pass it as a variable to the generation of pydeck (i.e. `pdk.Deck(mapbox_key=<mapbox-key-here>, ...)`.

In [ ]:
import pydeck as pdk

# for pydeck the attributes have to be flat
track_df['lat'] = track_df['geometry'].apply(lambda coord: coord.y)
track_df['lng'] = track_df['geometry'].apply(lambda coord: coord.x)
vis_df = pd.DataFrame(track_df)
vis_df['speed'] = vis_df['Speed.value']

# omit unit columns
vis_df_cols = [col for col in vis_df.columns if col.lower()[len(col)-4:len(col)] != 'unit']
vis_df = vis_df[vis_df_cols]

layer = pdk.Layer(
    'ScatterplotLayer',
    data=vis_df,
    get_position='[lng, lat]',
    auto_highlight=True,
    get_radius=10,          # Radius is given in meters
    get_fill_color='[speed < 20 ? 0 : (speed - 20)*8.5, speed < 50 ? 255 : 255 - (speed-50)*8.5, 0, 140]',  # Set an RGBA value for fill
    pickable=True
)

# Set the viewport location
view_state = pdk.ViewState(
    longitude=7.5963592529296875,
    latitude=51.96246168188569,
    zoom=10,
    min_zoom=5,
    max_zoom=15,
    pitch=40.5,
    bearing=-27.36)

r = pdk.Deck(
    width=200, 
    layers=[layer], 
    initial_view_state=view_state,
    mapbox_key='pk.eyJ1IjoienVyY3BoIiwiYSI6ImNrOHgxYWI1cDB0Ym0zbXBrc2VxdGVlYWkifQ.yAnuEOOC0WTKMeIkiPwQKQ'
)
r.to_html('tracks_muenster.html', iframe_width=900)

# Visualization of Single Track

Changed the attributes (latitude and longitude) to refer to the variable "some_track" created in cell 7, instead of showing the whole track data frame.

Redirected the viewport location to the coordinates of this track and enhanced the zoom to focus on the single track instead 
of the whole city of Muenster.

In [ ]:
import pydeck as pdk

# for pydeck the attributes have to be flat
some_track['lat'] = some_track['geometry'].apply(lambda coord: coord.y)
some_track['lng'] = some_track['geometry'].apply(lambda coord: coord.x)
vis_df = pd.DataFrame(some_track)
vis_df['speed'] = vis_df['Speed.value']

# omit unit columns
vis_df_cols = [col for col in vis_df.columns if col.lower()[len(col)-4:len(col)] != 'unit']
vis_df = vis_df[vis_df_cols]

layer = pdk.Layer(
    'ScatterplotLayer',
    data=vis_df,
    get_position='[lng, lat]',
    auto_highlight=True,
    get_radius=10,          # Radius is given in meters
    get_fill_color='[speed < 20 ? 0 : (speed - 20)*8.5, speed < 50 ? 255 : 255 - (speed-50)*8.5, 0, 140]',  # Set an RGBA value for fill
    pickable=True
)

# Set the viewport location
view_state = pdk.ViewState(
    longitude=7.614217519274591,
    latitude=51.94663734506707,
    zoom=13,
    min_zoom=5,
    max_zoom=15,
    pitch=40.5,
    bearing=-27.36)

r = pdk.Deck(
    width=200, 
    layers=[layer], 
    initial_view_state=view_state,
    mapbox_key='pk.eyJ1IjoienVyY3BoIiwiYSI6ImNrOHgxY2k2ZTEybTUzbW1oNXNsZ3hjdzYifQ.ykwDPZC1gvrOmMz6YhovAw'
)
r.to_html('tracks_muenster_single.html', iframe_width=900)